In [ ]:
#integration functions
import random
from skhep.math.vectors import LorentzVector, Vector3D
import math
self=model
def integrate(br, coupling, m0, m1, m2, m3, nsample):

    # prepare output
    particles, weights = [], []
    #create parent 4-vector
    p_mother=LorentzVector(0,0,0,m0)

    #integration boundary
    q2min,q2max = (m2+m3)**2,(m0-m1)**2
    mass = m3
    #numerical integration
    integral=0
    for i in range(nsample):

        #Get kinematic Variables
        q2 = random.uniform(q2min,q2max)
        q  = math.sqrt(q2)
        E2st = (q**2 - m2**2 + m3**2)/(2*q)
        E3st = (m0**2 - q**2 - m1**2)/(2*q)
        m232min = (E2st + E3st)**2 - (np.sqrt(E2st**2 - m3**2) + np.sqrt(E3st**2 - m1**2))**2
        m232max = (E2st + E3st)**2 - (np.sqrt(E2st**2 - m3**2) - np.sqrt(E3st**2 - m1**2))**2
        cthmax = (m232max + q**2 - m2**2 - m1**2)/(2*m0)
        cthmin = (m232min + q**2 - m2**2 - m1**2)/(2*m0)
        ENmax = (m232max + q**2 - m2**2 - m1**2)/(2*m0)
        ENmin = (m232min + q**2 - m2**2 - m1**2)/(2*m0)
        EN = random.uniform(ENmin,ENmax)
        #th = np.arccos(EN)


        #decay meson and V
        #cosQ =random.uniform(-1,1)
        #phiQ =random.uniform(-math.pi,math.pi)
        #cosM =random.uniform(-1.,1.)
        #phiM =random.uniform(-math.pi,math.pi)
        #branching fraction
        brval  = eval(br)
        #print("q val ",q," EN val ",EN," Br ",brval, " Br ",eval(br))
        brval *= (q2max-q2min)*(cthmax-cthmin)/float(nsample)

        #save
        #particles.append(p_3)
        weights.append(brval)
    #print('branching frac', brval)
    #return particles,weights
    return(sum(weights))

def integrate_q2EN(pid0,pid1,pid2,m3,coupling,dbr,nsample):
    m0=self.masses(f'{pid0}')
    m1=self.masses(f'{pid1}')
    m2=self.masses(f'{pid2}')
    integ=integrate(dbr, 1, m0, m1, m2,m3, nsample)
    return(integ)
    
def integrate_EN(br, coupling, m0, m1, m2, m3, nsample):
    #prepare output
    weights=[]
    mass = m3
    #numerical integration
    integral=0
    for i in range(nsample):
        #get energy bounds
        ENmin=m3
        ENmax=(m0**2+m3**2-m1**2)/(2*m0)
        EN = random.uniform(ENmin,ENmax)

        #branching fraction
        brval  = eval(br)
        brval *= (ENmax-ENmin)/float(nsample)

        #save
        weights.append(brval)
    return(sum(weights))


#pid0 is parent particle, pid1 is produced meson pid2 is the smaller particle
#
from matplotlib import pyplot as plt
self=model
def show_br_curve(br,pid0,pid1,pid2,integration,nsample=100):
    if integration=='br':
        coupling=1
        delm=.1
        mass0=0
        mass=0.01
        x=[]
        y=[]
        for n in range(1,int((3-mass0)/delm)):
            y.append(eval(br))
            x.append(mass)
            mass+=delm

        plt.plot(x,y)
        plt.xlim([0,3])
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")

    if integration=='dEN':
        dbr=br
        m0=self.masses(f"{pid0}")
        m1=self.masses(f"{pid1}")
        m2=self.masses(f"{pid2}")
        m3=0
        qmin,qmax=(m2+m3),(m0-m1)
        coupling=1
        delm=.1
        x=[]
        y=[]
        for n in range(1,30):
            y.append(integrate_EN(dbr, 1, m0, m1, m2,m3, nsample))
            x.append(m3)
            m3+=delm
        plt.xlim([0,2])
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")
        plt.plot(x,np.array(y))

    if integration=='dq2dEN':
        dbr=br
        m0=self.masses(f"{pid0}")
        m1=self.masses(f"{pid1}")
        m2=self.masses(f"{pid2}")
        coupling=1
        delm=.1
        x=[]
        y=[]
        m3=0
        for n in range(1,30):
            y.append(integrate_q2EN(pid0,pid1,pid2,m3,coupling,dbr,nsample))
            #y.append(integrate_pseud_3body(df,pid0,pid1,pid2,m3,1,nsample=nsample,channel="D"))
            x.append(m3)
            m3+=delm
        plt.ylim([0,max(y)])
        plt.xlabel(r"$m_N (GeV)$")
        plt.ylabel(r"Br")
        plt.plot(x,np.array(y))

    plt.show()


import pandas as pd

#n can be: 2 for 2-body and 3 for 3-body. type can be: 2 body regular, 2 body tau to pi, 2 body tau to K, 2 body tau to rho, 3 body pseudo, 3 body vector, 3 body tau
#pid0 is parent pid1 is produces meson pid2 is the smaller particle, usually the electron
#for 2 body pid0 is the parent and pid1 is the produced SM particle
def add_nbodydecay(pid0,pid1,pid2,energy='14',nsample=10,generator="Pythia8",n=2,type_decay="2 body regular",show_br=True):
    if n==2 and type_decay=="2 body regular":
        df=pd.DataFrame([],columns=['particle','pid0','tauH (sec)','MH (GeV)','VH (unitless)','fH (GeV)','process'])
        particles=[['D^+','411',1040*10**(-15),1.8696,.221,.2226,'D^+ -> e^+ + N'],['D^+_s','431',500*10**(-15),1.96847,0.987,.2801, 'D_s^+ -> e^+ + N'],['B^+','521',1.638*10**(-12),5.27934,3.82*10**(-3),0.187,'B^+ -> e^+ + N'],['B^+_c','541',0.510*10**(-12),6.2749,41*10**(-3),.434, 'B^+_c -> e^+ + N']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        br=hnl.get_2body_br(pid0,pid1)
        model.add_production_2bodydecay(
        pid0 = f"{pid0}",
        pid1 = f"{pid1}",
        br=br,
        generator = generator,
        energy = energy,
        nsample = nsample
        )
        if show_br==True:
            show_br_curve(br,pid0,pid1=pid1,pid2=pid2,integration='br',nsample=nsample)
        #show_br_curve(df,br,pid0,pid1=None,pid2=pid2,decay_type='2 body',nsample=nsample)
        
    if n==2 and type_decay=="2 body tau":
        df=pd.DataFrame([],columns=['pid0','pid1','process'])
        particles=[['15','-211','tau^- -> pi^- + N'],['15','-321','tau^- -> K^- + N'],['15','-213','tau^- -> rho^- + N']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]

        if pid1=="-213" or pid1=="213":
            br=hnl.get_2body_br_tau_rho(pid0,pid1)
        else:
            br=hnl.get_2body_br_tau_H(pid0,pid1)

        model.add_production_2bodydecay(
            pid0 = f"{pid0}",
            pid1 = f"{pid1}",
            br=br,
            generator = generator,
            energy = energy,
            nsample = nsample
        )
        if show_br==True:
            show_br_curve(br,pid0,pid1=pid1,pid2=pid2,integration='br',nsample=nsample)
        
    #here pid1 is the electron (lepton) now and pid2 is the neutrino
    if n==3 and type_decay=="3 body tau":
        df=pd.DataFrame([],columns=['pid0','pid1','pid2','process'])
        particles=[['15','11','18','tau^- -> e^- + nu_tau']]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_tau(pid0,pid1,pid2)
        model.add_production_3bodydecay(
            label= "5_di",
            pid0 = pid0,
            pid1 = pid1,
            pid2 = pid2,
            br = dbr,
            generator ="pythia8",
            energy = energy,
            nsample = nsample,
            scaling = 0, 
        )
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dEN",nsample=nsample)
        
    if n==3 and type_decay=="3 body pseudo":
        df=pd.DataFrame([],columns=['process','pid0','pid1','tauH (sec)','VHHp'])
        particles=[['D^+ -> \bar{K}^0 + l + N','411','-311',1040*10**(-15),0.987],
        ['D^- -> \bar{K}^0 + l + N','-411','-311',1040*10**(-15),0.987],
        ['D^0 -> K^- + l + N','421','-321',410.1*10**(-15),0.997],
        ['B^+ -> \bar{D}^0 + l + N','521','421',1.638*10**(-12),41*10**(-3)],
        ['B^0 -> D^- + l + N','511','-411',1.519*10**-12,42.2*10**-3],
        ['B^0_s -> D^-_s + l + N','531','-431',1.515*10**-12,42.2*10**-3],
        ['B^+_c -> B^0 + l + N','541','511',0.510*10**-12,0.221],
        ['B^+_c -> B^0_s + l + N','541','531',0.510*10**-12,0.987]]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_pseudoscalar(pid0,pid1,pid2)
        model.add_production_3bodydecay(
        label= "5_di",
        pid0 = pid0,
        pid1 = pid1,
        pid2 = pid2,
        br = dbr,
        generator = "Pythia8",
        energy = energy,
        nsample = nsample,
        scaling = 0)
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dq2dEN",nsample=nsample)

    if n==3 and type_decay=="3 body vector":
        df=pd.DataFrame([],columns=['process','pid0','pid1','tauH (sec)','VHHp'])
        particles=[['D^0 -> K*^- + e^+ + N','421','-323',410.1*10**(-15),0.967],['B^+ -> \bar{D}*^0 + e^+ + N','521','-423',1.638*10**-12,41*10**-3],['B^0 -> D*^- + e^+ + N','511','-413',1.519*10**-12,41*10**-3],['B^0_s -> D^*_s^- + e^+ + N','531','-433',1.515*10**-12,41*10**-3],['B^+_c -> B*^0 + e^+ + N','541','513',.510*10**-12,.221],['B^+_c -> B^*_s^0+ e^+ + N','541','533',.510*10**-12,41*10**-3]]
        for n in range(len(particles)):
            df.loc[len(df)]=particles[n]
        dbr=hnl.get_3body_dbr_vector(pid0,pid1,pid2)
        model.add_production_3bodydecay(
        label= "5_di",
        pid0 = pid0,
        pid1 = pid1,
        pid2 = pid2,
        br = dbr,
        generator = "Pythia8",
        energy = energy,
        nsample = nsample,
        scaling = 0)
        if show_br==True:
            show_br_curve(dbr,pid0,pid1,pid2,integration="dq2dEN",nsample=nsample)


#for testing functions
#for 2 body decays
df=pd.DataFrame([],columns=['particle','pid0','tauH (sec)','MH (GeV)','VH (unitless)','fH (GeV)','process'])
particles=[['D^+','411',1040*10**(-15),1.8696,.221,.2226,'D^+ -> e^+ + N'],['D^+_s','431',500*10**(-15),1.96847,0.987,.2801, 'D_s^+ -> e^+ + N'],['B^+','521',1.638*10**(-12),5.27934,3.82*10**(-3),0.187,'B^+ -> e^+ + N'],['B^+_c','541',0.510*10**(-12),6.2749,41*10**(-3),.434, 'B^+_c -> e^+ + N']]
for n in range(len(particles)):
    df.loc[len(df)]=particles[n]

#for 2 body decays tau
df_t=pd.DataFrame([],columns=['pid0','pid1','process'])
particles=[['15','-211','tau^- -> pi^- + N'],['15','-321','tau^- -> K^- + N'],['15','-213','tau^- -> rho^- + N']]
for n in range(len(particles)):
    df_t.loc[len(df_t)]=particles[n]

#for 3 body decays tau
df_3t=pd.DataFrame([],columns=['pid0','pid1','pid2','process'])
particles=[['15','11','18','tau^- -> e^- + nu_tau + N'],['15','11','-12','tau^- -> e^- + \bar{nu}_e + N']]
for n in range(len(particles)):
    df_3t.loc[len(df_3t)]=particles[n]

#for 3 body pseudo
df_D=pd.DataFrame([],columns=['particle','pid0','pid1','tauH (sec)','VHHp'])
particles=[['D^+ -> \bar{K}^0 + l + N','411','-311',1040*10**(-15),0.987],
['D^- -> \bar{K}^0 + l + N','-411','-311',1040*10**(-15),0.987],
['D^0 -> K^- + l + N','421','-321',410.1*10**(-15),0.997],
['B^+ -> \bar{D}^0 + l + N','521','421',1.638*10**(-12),41*10**(-3)],
['B^0 -> D^- + l + N','511','-411',1.519*10**-12,42.2*10**-3],
['B^0_s -> D^-_s + l + N','531','-431',1.515*10**-12,42.2*10**-3],
['B^+_c -> B^0 + l + N','541','511',0.510*10**-12,0.221],
['B^+_c -> B^0_s + l + N','541','531',0.510*10**-12,0.987]]
for n in range(len(particles)):
    df_D.loc[len(df_D)]=particles[n]

#for 3 body vector
df=pd.DataFrame([],columns=['particle','pid0','pid1','tauH (sec)','VHHp'])
particles=[['D^0 -> K*^- + e^+ + N','421','-323',410.1*10**(-15),0.967],
['B^+ -> \bar{D}*^0 + e^+ + N','521','-423',1.638*10**-12,41*10**-3],
['B^0 -> D*^- + e^+ + N','511','-413',1.519*10**-12,41*10**-3],
['B^0_s -> D^*_s^- + e^+ + N','531','-433',1.515*10**-12,41*10**-3],
['B^+_c -> B*^0 + e^+ + N','541','513',.510*10**-12,.221],
['B^+_c -> B^*_s^0+ e^+ + N','541','533',.510*10**-12,41*10**-3]]
for n in range(len(particles)):
    df.loc[len(df)]=particles[n]

pid0='421'
pid1='-323'
pid2='-11'

add_nbodydecay(pid0,pid1,pid2,energy='14',nsample=100,generator="Pythia8",n=3,type_decay="3 body vector",show_br=True)

In [ ]:
#HNL decay attempts
#HNL decays 2 body decays
#pid1 is the meson pid2 is the smaller particle either lepton or neutrino
#from HNL_decays_1
#fH is called the decay constant
self=model
SecToGev=1./(6.582122*pow(10.,-25.))
pid1="111"
tauN=10**-5*SecToGev
GF=1.166378*10**(-5) #GeV^(-2)
coupling=1
mass=1
pid1="111"
pid2="12" #electron neutrino
br_pi_nu=f"tauN*(coupling**2*GF**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_pi_nu))
pid1="411"
pid2="11"
br_H_l=f"(tauN*coupling**2*GF**2*hnl.VH(pid1)**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*((1-self.masses(pid2)**2/mass**2)**2-(self.masses(pid1)**2/mass**2)*(1+self.masses(pid2)**2/mass**2))*np.sqrt((1-((self.masses(pid1)-self.masses(pid2))**2/mass**2))*(1-((self.masses(pid1)+self.masses(pid2))**2/mass**2)))"
print(eval(br_H_l))
pid1="221"
pid2="12"
br_eta_nu=f"(tauN*coupling**2*GF**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_eta_nu))
pid1="331"
pid2="12"
br_etap_nu=f"(tauN*coupling**2*GF**2*hnl.fH(pid1)**2*mass**3/(32*np.pi))*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_etap_nu))
pid1="213"
pid2="11"
grho=0.102 #check this?
br_rho_l=f"(tauN*coupling**2*grho**2*GF**2*hnl.VH(pid1)**2*mass**3/(8*np.pi))*((1-self.masses(pid2)**2/mass**2)**2+(self.masses(pid1)**2/mass**2)*(1+((self.masses(pid2)**2-2*self.masses(pid1)**2)/mass**2))*np.sqrt((1-((self.masses(pid1)-self.masses(pid2))**2/mass**2))*(1-((self.masses(pid1)+self.masses(pid2))**2/mass**2))))"
print(eval(br_rho_l))
pid1="113"
pid2="12"
br_rho0_nu=f"(tauN*coupling**2*grho**2*GF**2*mass**3/(16*np.pi*self.masses(pid1)**2))*(1+2*self.masses(pid1)**2/mass**2)*(1-self.masses(pid1)**2/mass**2)**2"
print(eval(br_rho0_nu))

In [ ]:
#HNL decays attempt 2
#HNL decays redo.
#refer to the file All_HNL_decays?
from scipy.integrate import quad
leptons=['11','13','15']
GF=1.166378*10**(-5)
pid1="411"
br_l_P=f"(GF**2/(16*np.pi))*hnl.fH(pid1)**2*hnl.VH(pid1)**2*coupling**2*mass**3"     #N->l^- P^+, P pseudoscalar
br_nul_P=f"(GF**2/(64*np.pi))*hnl.fH(pid1)**2*coupling**2*mass**3"           #N->nu_l P^0
br_l_V=f"(GF**2/(16*np.pi))*hnl.fH(pid1)**2*hnl.VH(pid1)**2*coupling**2*mass**3"        #N->l^- V^+, V is a vector
br_nul_V=f"(GF**2/(2*np.pi))*hnl.kV(pid1)**2*hnl.fH(pid1)**2*coupling**2*mass**3"       #N->nu_l V^0
br_l1_l2_nul2=f"(GF**2/(192*np.pi**3))*coupling**2*mass**5"         #N->l1^- l_2^+ nul2
##############
#xl2=massl2/mN; l1=e,mu,tau=1,2,3; l2=e,mu,tau=1,2,3
def bra_nul1_l2_l2(mass,l1,l2):
    self=model
    xl2=self.masses('11')/mass
    x=0
    xw=0.231
    glL=-1/2+xw
    glR=xw
    GF=1.166378*10**(-5)
    lamda=lambda a,b,c: a**2+b**2+c**2-2*a*b-2*b*c-2*c*a
    I1_integrand=lambda s,xl2,x: (12/s)*(s-0*x**2-xl2**2)*(1+xl2**2-s)*np.sqrt(lamda(s,0*x**2,xl2**2))*np.sqrt(lamda(1,s,xl2**2))
    I1=quad(I1_integrand,(x*0+xl2)**2,(1-xl2)**2,args=(xl2,0))[0]
    I2_integrand=lambda s,xl2,x: (24*xl2*xl2/s)*(1-0*x**2-s)*np.sqrt(lamda(s,xl2**2,xl2**2))*np.sqrt(lamda(1,s,0*x**2))
    I2=quad(I2_integrand,(xl2+xl2)**2,(1-x*0)**2,args=(xl2,0))[0]
    delta=lambda l1,l2: 1 if l1==l2 else 0
    br_nul1_l2_l2=f"({GF}**2/(96*np.pi**3))*coupling**2*mass**5*(({glL}*{glR}+{delta(l1,l2)}*{glR})*{I2}+({glL}**2+{glR}**2+{delta(l1,l2)}*(1+2*{glL}))*{I1})"
    return(br_nul1_l2_l2)
    #return(I1,I2,delta(l1,l2),f"{delta(l1,l2)}")

##############
br_nul1_nu_nu=f"(GF**2/(96*np.pi**3))*coupling**2*mass**5"          #\sum_{l_2 = e}^{\tau} \Gamma(N->nu_l1 nu_l2 \bar{nu}_l2)


tau_N=f"6.58*10**-14*coupling**-2"
pseudoscalars_symbol=['K^0','K+', 'barK0','pi0', 'pi+', 'eta', "eta'"]
pseudoscalars=["311","-311","321","-321","111","211","-211","221","331"]            #[K^0,K^{\pm}, \bar{K}^0,\pi^0, \pi^{\pm}, \eta, \eta"] #what about D_s^+, D^+, etc.?
pseudoscalars=["311","321","111","211","221","331"] 
pseudoscalar_charged_symbols=["D^+","D^-" ,"B^+" ,"B^-" ,"B^+_c","B_c^-"]
pseudoscalars_charged=       ["411","-411","521" ,"-521","541"  ,"-541" ]
pseudoscalar_neutral_symbols=["D^0","B^0","B_s^0"]
pseudoscalars_neutral=       ["421","511","531"  ] #not sure what to do with  neutral mesons, since they dont have a ckm matrix elements, same with the light mesons
vectors_symbol=['K*0', 'barK*0', 'K*+', 'rho0', 'rho+', 'omega','phi']
vector_charged_symbols=["K^+","K^-" ]
vectors_charged=       ["321","-321"] #I believe these are the 'heavy vector mesons, not including rho0 omega phi, etc.; I took out "213","-213" (rho^+) since it doesnt have an fH it has a grho.
vectors_neutral_symbols=["K^{*0}","\bar{K}^{*0}","D^{*0}","\bar{D}^{*0}"]
vectors_neutral=        ["313"   ,"-313"        ,"423"   ,"-423"] #maybe add 443 J/psi, I just dont have fH for that one
vectors_idk=["113","223","333","443"]
#need to add br for every possible combination of l, p, nul,p etc.
#lets get it up and running for just 1
coupling=1
#the combination of coupling=1 and the extra factor*6.58*10**-14 accounts for tau_N
#br does not depend on the coupling


#for the decays N->l^{\mp} P^{\pm}
for n in range(len(pseudoscalars_charged)):
    pid1=pseudoscalars_charged[n]
    write_m_br_file(br_l_P+"*6.58*10**-14",filename=f"model/br/br_l_{pid1}.txt")

print("charged pseudoscalars done...")

#for the decays N->nu_l P^0
for n in range(len(pseudoscalars_neutral)):
    pid1=pseudoscalars_neutral[n]
    write_m_br_file(br_nul_P+"*6.58*10**-14",filename=f"model/br/br_nul_{pid1}.txt")

print("neutral pseudoscalars done")

##############
#for the decays N->l^{\mp} V^{\pm}
for n in range(len(vectors_charged)):
    pid1=vectors_charged[n]
    print(hnl.VH(pid1))
    print(hnl.fH(pid1))
    write_m_br_file(br_l_V+"*6.58*10**-14",filename=f"model/br/br_l_{pid1}.txt")

print("charged vectors done...")

#for the decays N->nu_l V^0
for n in range(len(vectors_neutral)):
    pid1=vectors_neutral[n]
    #print(hnl.VH(pid1))
    print(hnl.fH(pid1))
    write_m_br_file(br_nul_V+"*6.58*10**-14",filename=f"model/br/br_nul_{pid1}.txt")

print("neutral vectors done...")

#for the decays N->l_1^{\mp} l_2^{\pm} nu_l2
write_m_br_file(br_l1_l2_nul2+"*6.58*10**-14",filename=f"model/br/br_l1_l2_nul2.txt")

print("leptons neutrinos done 1...")

#for the decays N->nu_l1 l_2^{\mp} l_2^{\pm}
xl2=1
l1=1
l2=2
#bra_nul1_l2_l2(xl2,coupling,mass,l1,l2)
write_m_brm_file(filename=f"model/br/br_nu1_l2_l2.txt")


print("leptons neutrinos done 2...")

#for the decays N->sum(\Gamma(N->nu_l1 nu_l2 \bar{nu}_l2))
write_m_br_file(br_nul1_nu_nu+"*6.58*10**-14",filename=f"model/br/br_nul1_nu_nu.txt")

print("neutrinos done...")

write_m_ctau_file(filename="model/ctau.txt")

print("ctau file created...")